In [237]:
#stern gerlach experiment

In [238]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import Node
from sequence.components.detector import Detector
from sequence.components.photon import Photon
from sequence.components.memory import Memory
from sequence.kernel.process import Process
from sequence.kernel.quantum_state import State
from sequence.kernel.event import Event
from sequence.components.optical_channel import QuantumChannel
import numpy as np

In [239]:
#state = State(complex(1/np.sqrt(2)), complex(1/np.sqrt(2)))
#photon = Photon("photon", self.timeline, quantum_state=state)
#append this to the init after fixing everything else

making the nodes that have the appropriate components for stern gerlach

In [240]:
class sendernode(Node):
    def __init__(self, name, timeline, seed=None, gate_fid = 1, meas_fid = 1):
        super().__init__(name, timeline, seed, gate_fid, meas_fid)
        memo_name = "{name}.name" 
        memo = Memory(memo_name, timeline, 1, 500, 1, -1, 500)
        self.memo = memo
        memo.add_receiver(self)
        self.add_component(memo)
        qubit_name = f"{name}.qubit"
        self.qubit = Photon(qubit_name, timeline, 500, self.memo)
        alpha = (1/np.sqrt(2))
        beta = (1/np.sqrt(2))
        state_vector=[complex(alpha), complex(beta)]
        self.qubit.set_state(state_vector)

    #def send_qubit(self, qubit, **kwargs):
    #   dst = kwargs["dst"]
    #   super().send_qubit(dst, self.qubit)

    def emit(self, dst): #memo : Memory)
        photon = self.qubit
        #state = State(complex(1/np.sqrt(2)), complex(1/np.sqrt(2)))
        #photon.set_state(state)
        #memo.update_state(state)
        #self.receive_qubit(memo, photon)
        self.send_qubit(dst,photon) #have to eliminate initialising photon inside the emit function, maybe make it an arg in the init.

In [241]:
class detector(Node):
    def __init__(self, name, timeline, seed=None, gate_fid = 1, meas_fid = 1):
        super().__init__(name, timeline, seed, gate_fid, meas_fid)
        self.results = []
        detector_name = f"{name}_name" 
        det = Detector(detector_name, timeline, 1)
        self.add_component(det)
        det.add_receiver(self)
        self.det = det
    def receive_qubit(self, src, qubit : Photon):
        prob = np.abs(qubit.quantum_state.state) ** 2
        result = np.random.choice(2, p=prob)
        self.results.append(result)
        ...
        print(f"received qubit from {src} is : {1 if result == 1 else 0}")

In [242]:
timeline = Timeline()
sender = sendernode("sender", timeline)
observe = detector("detector", timeline)
sender.set_seed(0)
observe.set_seed(1)
qc = QuantumChannel("qc", timeline, 0, 500)
qc.set_ends(sender, observe.name)  
timeline.init()
sender.emit(dst=observe.name)
observe.receive_qubit(sender, sender.qubit)
process1 = Process(sender, "emit", [(observe.name)]) #might not work due to photon being initialised inside the function itself 
process2 = Process(observe, "receive_qubit", [sender, sender.qubit])
event1 = Event(0, process1)
event2 = Event(1,process2)
timeline.schedule(event1)
timeline.schedule(event2)
timeline.run()

received qubit from sender is : 0
received qubit from sender is : 0
received qubit from sender is : 1
received qubit from sender is : 0


In [243]:
print((sender.qubit.quantum_state.state)[0]**2)

(0.4999999999999999+0j)
